<a href="https://colab.research.google.com/github/clumsyarm/drone_uav/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import glob
import os

def process_and_label_uav_data(input_dir, output_file, fault_code, start_sec, end_sec):
    """
    Merges UAV sensors and labels a specific sensor fault.
    The start_sec and end_sec are manually entered in seconds.
    """

    # 1. Define patterns for sensor files
    patterns = {
        'accel': '*sensor_accel_*.csv',
        'gyro': '*sensor_gyro_*.csv',
        'mag': '*sensor_mag_*.csv',
        'baro': '*sensor_baro_*.csv',
        'gps': '*sensor_gps_*.csv'
    }

    # Columns relevant for Anomaly Detection
    sensor_cols = {
        'accel': ['x', 'y', 'z', 'temperature', 'error_count', 'clip_counter[0]', 'clip_counter[1]', 'clip_counter[2]'],
        'gyro': ['x', 'y', 'z'],
        'mag': ['x', 'y', 'z'],
        'baro': ['pressure', 'temperature'],
        'gps': ['lat', 'lon', 'alt', 'eph', 'epv', 'vel_m_s', 'satellites_used']
    }

    # Fault column mapping
    fault_mapping = {1: 'accel_fault', 2: 'baro_fault', 3: 'gps_fault', 4: 'gyro_fault', 5: 'mag_fault'}

    dfs = {}

    # 2. Load and Prepare Sensor Data
    for sensor, pattern in patterns.items():
        files = glob.glob(os.path.join(input_dir, pattern))
        if not files:
            print(f"Warning: No file found for {sensor}")
            continue

        df = pd.read_csv(files[0])
        cols = ['timestamp'] + [c for c in sensor_cols[sensor] if c in df.columns]
        df = df[cols].copy()
        df.columns = ['timestamp'] + [f"{sensor}_{c}" for c in df.columns if c != 'timestamp']
        dfs[sensor] = df.sort_values('timestamp')

    if 'accel' not in dfs:
        print("Error: Accelerometer file not found. It is required as the time base.")
        return

    # 3. Synchronized Merge (Nearest Neighbor)
    merged_df = dfs['accel']
    for sensor in [s for s in dfs.keys() if s != 'accel']:
        print(f"Merging {sensor}...")
        merged_df = pd.merge_asof(merged_df, dfs[sensor], on='timestamp', direction='nearest')

    # 4. CONVERT MANUAL SECONDS TO MICROSECONDS
    # We find the first timestamp of the flight and add your seconds to it.
    base_timestamp = merged_df['timestamp'].min()

    start_t_micro = int(base_timestamp + (start_sec * 1_000_000))
    end_t_micro = int(base_timestamp + (end_sec * 1_000_000))

    print(f"\n--- Time Conversion ---")
    print(f"Log Start Timestamp: {base_timestamp}")
    print(f"Fault Start (microsec): {start_t_micro} ({start_sec}s from start)")
    print(f"Fault End   (microsec): {end_t_micro} ({end_sec}s from start)")

    # 5. Multi-Column Fault Labeling
    for col_name in fault_mapping.values():
        merged_df[col_name] = 0

    target_fault_col = fault_mapping.get(fault_code)
    if target_fault_col:
        mask = (merged_df['timestamp'] >= start_t_micro) & (merged_df['timestamp'] <= end_t_micro)
        merged_df.loc[mask, target_fault_col] = 1
        print(f"Successfully labeled {mask.sum()} rows in column '{target_fault_col}'")
    else:
        print("Invalid fault code. No labeling applied.")

    # 6. Save the final CSV
    merged_df.to_csv(output_file, index=False)
    print(f"Done! Saved merged and labeled file to: {output_file}")

# ==========================================
# USER CONFIGURATION AREA (Enter your values here)
# ==========================================

# 1. The folder where your CSV files are located
FOLDER_PATH = "."

# 2. Name for your merged output file
OUTPUT_NAME = "merged_flight_4_labeled.csv"

# 3. Which sensor had the fault?
# 1: Accel, 2: Baro, 3: GPS, 4: Gyro, 5: Mag
SELECTED_FAULT_CODE = 1

# 4. MANUALLY ENTER TIME IN SECONDS (from your TestInfo file)
START_SEC = 23.0  # e.g., Fault injection time
END_SEC = 33.0    # e.g., Test end time

# Run the process
process_and_label_uav_data(FOLDER_PATH, OUTPUT_NAME, SELECTED_FAULT_CODE, START_SEC, END_SEC)

Merging gyro...
Merging mag...
Merging baro...
Merging gps...

--- Time Conversion ---
Log Start Timestamp: 99976004
Fault Start (microsec): 122976004 (23.0s from start)
Fault End   (microsec): 132976004 (33.0s from start)
Successfully labeled 100 rows in column 'accel_fault'
Done! Saved merged and labeled file to: merged_flight_4_labeled.csv
